In [4]:
from datetime import datetime as t
import paho.mqtt.client as mqtt
import json
import mqtt_config as config

In [2]:
client = mqtt.Client('config')
listen = config.domain + 'rawdata/' + config.Location + '/' + config.pumpName
broker = config.mqtt_broker
publish = config.domain + 'edits/' + config.Location + '/' + config.pumpName
client.connect(broker, keepalive=60)

0

In [15]:
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT Broker!")
        print(f"Listening on topic: {listen}")
    else:
        print(f"Failed to connect, return code {rc}", "Error\t")

def on_disconnect(client, userdata, rc):
    print(f"Unexpected disconnection due to {rc}")
    while True:
        conn = client.connect(broker, keepalive=60)
        print("Reconnecting...")
        if conn:
            break
        else:
            continue
        time.sleep(5)

def on_message(client, userdata, msg):
    x = msg.payload
    command = json.loads(x)
    print(f"Recieved write command {command}")

Change Values

In [21]:
write_message = json.dumps({
    'register': [2, 0x01],
    'bit': [0, 0x01]
})
try:
    client.connect(broker, keepalive=60)
    client.publish(publish, write_message, qos=1)
    print(f"published {write_message} to {publish} at {t.now()}")
except Exception as e:
    print(e)
try:
    client.loop_start()
    client.on_connect = on_connect
    client.on_message = on_message
    client.on_disconnect = on_disconnect
    client.subscribe(publish)
    client.loop_stop()
except Exception as r:
    print(f'There was an issue sending data because {r}.. Reconnecting')
    connection = client.connect(broker)

published {"register": [2, 1], "bit": [0, 1]} to buckman/ackumen/smartAssets/edits/Chennai/Ch_dda003 at 2022-10-02 21:29:57.492276
Connected to MQTT Broker!
Listening on topic: buckman/ackumen/smartAssets/rawdata/Chennai/Ch_dda003
Recieved write command {'register': [2, 1], 'bit': [0, 1]}


Read registers to check changes

In [22]:
write_message = json.dumps({
    'register': [1,2,100,101,102,103,104,105,106,107,200,201,202,203,204,205,206,207],
})
try:
    client.connect(broker, keepalive=60)
    client.publish(publish, write_message, qos=1)
    print(f"published {write_message} to {publish} at {t.now()}")
except Exception as e:
    print(e)
try:
    client.loop_start()
    client.on_connect = on_connect
    client.on_message = on_message
    client.on_disconnect = on_disconnect
    client.subscribe(listen)
    client.loop_stop()
except Exception as r:
    print(f'There was an issue sending data because {r}.. Reconnecting')
    connection = client.connect(broker)

published {"register": [1, 2, 100, 101, 102, 103, 104, 105, 106, 107, 200, 201, 202, 203, 204, 205, 206, 207]} to buckman/ackumen/smartAssets/edits/Chennai/Ch_dda003 at 2022-10-02 21:31:47.153308
Connected to MQTT Broker!
Listening on topic: buckman/ackumen/smartAssets/rawdata/Chennai/Ch_dda003
